### P4.1 Symulacja ruchu kulek na siatce sprężynowej

Rozważyć ruch kulek zamieszczonych na siatce sprężynowej (2D lub 3D).

- Utworzyć animację kulek umieszczonych na siatce.
- Rozważyć różne współczynniki oporu r
- Rozważyć dodatkową siłę grawitacji
- Rozważyć ramę siatki rozmiaru n x m

$m_i x_i'' = \sum_{j} \vec{F}_{i,j} + \vec{R}_{i}$

gdzie

$\vec{F}_{i,j}=\frac{(\vec{x}_{j}-\vec{x}_{i})}{d_{i,j}}\cdot(d_{i,j}-l)\cdot k$

$\vec{R}_{i}=-r\cdot ||\vec{x}_i'||^{\alpha}_2 \cdot \frac{\vec{x}_i'}{||\vec{x}_i'||_2}$

$d_{i,j} = ||\vec{x}_i-\vec{x}_j||_2$  –  odległość kulek

$l$  –  długość sprężynek w spoczynku

oraz $\alpha$ = 2, 1

In [1]:
%matplotlib inline
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time

def solve_vec(method, f, y0, rng, h):
    (K, N, M, D) = y0.shape
    t0, tn = rng
    
    t = np.arange(t0,tn,h)
    y = np.zeros((len(t), K, N, M, D), np.float64)
    y[0] = y0
    
    for ti in range(1, len(t)):
        y[ti] = method(f, t[ti-1], y[ti-1], h)
    return t, y

def euler_method(f, t, y, h):
    return y + h * f(t, y)

def midpoint_method(f, t, y, h):
    K1 = f(t, y)
    K2 = f(t+h*0.5, y+(h*0.5*K1))
    return y + h * K2

def RK4_method(f, t, y, h):
    K1 = f(t, y)
    K2 = f(t+h*0.5, y+(h*0.5*K1))
    K3 = f(t+h*0.5, y+(h*0.5*K2))
    K4 = f(t+h, y+(h*K3))
    return y + h*(K1/6+K2/3+K3/3+K4/6)

In [2]:
def getNeighbours(i,j,x):
    neighbours = []
    if i-1>=0:
        neighbours.append(x[i-1,j])
    if i+1<x.shape[0]:
        neighbours.append(x[i+1,j])
    if j-1>=0:
        neighbours.append(x[i,j-1])
    if j+1<x.shape[1]:
        neighbours.append(x[i,j+1])
    return neighbours
    
# sila F dla kulki (i,j)
def F(i, j, x, k, l):
    f = np.array([0,0], np.float64)
    if(i<=0 or j<=0 or i>=x.shape[0]-1 or j>=x.shape[1]-1):
        return f
    
    x_ij = x[i,j]
    for x_n in getNeighbours(i,j,x):
        d = np.linalg.norm(x_ij-x_n)
        f += ((x_n - x_ij)/d)*(d-l)*k
    return f

# sila R dla kulki (i,j)
def R(i, j, xp, r, alpha):
    xp_ij = xp[i,j]
    if(i<=0 or j<=0 or i>=xp.shape[0]-1 or j>=xp.shape[1]-1 or np.linalg.norm(xp_ij) == 0.):
        return np.array([0, 0], np.float64)
    
    f = (-r) * (np.linalg.norm(xp_ij)**alpha) * (xp_ij/np.linalg.norm(xp_ij))
    return f

N = 11
M = 21
r = 0.5
alpha = 1
k = 30
l = 0.2
m = np.array([[1 for i in range(M+2)] for j in range(N+2)], np.float64)

def f(t, y):
    (_, N, M, _) = y.shape
    values = np.zeros_like(y)
    for i in range(N):
        for j in range(M):
            values[:,i,j] = np.array([y[1,i,j], ( F(i,j,y[0],k,l) + R(i,j,y[1],r,alpha) ) / m[i,j]])
    return values

x0 = np.array([ [ [l*j, l*i] for i in range(M+2) ] for j in range(N+2) ], np.float64)
xd0 = np.zeros_like(x0)
x0[N/2+1, M/2+1] += np.array([0.19,0.19])

y0 = np.array([x0, xd0])

rng = (0,10)
h = 1/30.

In [3]:
def render_balls(pos, r, l, shape):
    img = np.ones(shape)*[32,40,48]
    (N, M, two) = pos.shape
    for i in range(1,N-1):
        for j in range(1,M-1):
            center = (int(pos[i,j,0]), int(pos[i,j,1]))
            neighbours = getNeighbours(i,j,pos)
            for pos_n in neighbours:
                # color in order to tension
                tension = (l-abs(np.linalg.norm(np.array(center)-np.array(pos_n))))*(255./l)
                color = [
                    128+np.clip(-tension,-127,127),
                    128+np.clip(-tension,-127,127),
                    128+np.clip(tension,-127,127)
                ]
                cv2.line(img, center, (int(pos_n[0]), int(pos_n[1])), color, 2)
    
    for i in range(1,N-1):
        for j in range(1,M-1):
            center = (int(pos[i,j,0]), int(pos[i,j,1]))
            cv2.circle(img, center, int(r[i,j]), [82,16,213], -1)
            cv2.circle(img, (center[0]+int(r[i,j]/3),center[1]-int(r[i,j]/3)), int(r[i,j])/3, [235,240,245], -1)
    return img

def balls_animation(t, x, m, l, max_size, pre=''):
    (_, N, M, _) = x.shape
    
    end_pos_n, end_pos_m = x[0,-1,-1,0], x[0,-1,-1,1]
    if end_pos_m > end_pos_n:
        frame_size = (int((end_pos_n/end_pos_m)*max_size), max_size)
        scale = max_size/end_pos_m
    else:
        frame_size = (max_size, int((end_pos_m/end_pos_n)*max_size))
        scale = max_size/end_pos_n
    
    shape = (frame_size[1], frame_size[0], 3)
    video = cv2.VideoWriter(
        'output/v_'+pre+'_'+str(time.time())+'.avi',
        cv2.VideoWriter_fourcc(*'MJPG'),
        len(t)//t[-1],frame_size)
    
    max_radius = 6
    r = m/np.max(m)
    r = (r**0.5)*max_radius
    for i in range(len(t)):
        frame = render_balls(x[i]*scale, r, l*scale, shape)
        video.write(frame.astype(np.uint8))
        
    video.release()

In [4]:
t, y = solve_vec(midpoint_method, f, y0, rng, h)
balls_animation(t, y[:,0], m, l, 1200, 'midpoint')

In [5]:
t, y = solve_vec(RK4_method, f, y0, rng, h)
balls_animation(t, y[:,0], m, l, 1200, 'RK4')